# Model Project: The Agents Labour Supply Decision


In present project we seek to model the supply of labour by studying a two-good static problem, where agents value leisure and consumption. As an extension for the base model we impose a progressive tax system, which creates kinks in the budget constraint. We then show, how this may create bunching around tax bracket cut-offs.

The basic problem of the agents is:


$$\begin{aligned}  \max_{c,l}\ \ &u(c,l) \quad s.t. \ \ R_0 = wl + c  \quad c,l\geq0\\
 &\dfrac{\partial u}{\partial a}= u_a>0 \quad  \wedge \quad \dfrac{\partial^2 u}{\partial a^2}=u_{aa}<0,  \qquad a=c,l \\
 &R_0=wT + A_0 
 \end{aligned} $$
 

Where $R_0$ is the income potential of the agent, $c$ the consumption which is a numeraire, $l$ is the amount of leisure consumed with the associated oppurtunity cost of being the wage rate, $w$. We assume marginal utility of consumption and leisure are positive but decreasing. Further we for now assume the income potential is the sum of wage earnings from working your entire time endowment, $T$, and non-labour income, $A_0$. The problem is framed such that the agent at default suplies all his time as labour, giving her/him an income of $R_0$. The agent now has to decide how much leisure she/he will 'buy back', considering the consumption potential which is implied.

Dependencies:

In [1]:
import numpy as np
import scipy as sp
from scipy import linalg
from scipy import optimize
from scipy.optimize import Bounds
import sympy as sm

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

The Langrangian for the problem:
$$ \mathcal{L}(c,l; \lambda)= u(c,l)\ + \ \lambda( R_0 - wl - c) $$ 

For which the first order conditions with regards to $c$ and $l$ implies an optimum at $\dfrac{u_l}{u_c}=w$ as the constraint is binding.
We will solve this problem using a numerical optimizer, which will allow for easy handling of our extension to this basic model. However to initially test our program, we will compare the analytical solution for a particular set of parameters to the optimizer.
## Example: Cobb Douglas utility
For a Cobb-Douglas utility functions, $u(c,l)= c^\alpha l^{1-\alpha}$, the solution Marshallian demand functions are:
$$c^*= w l \left(\dfrac{\alpha}{1-\alpha}\right)=R_0 \alpha \qquad l^*=R_0 (1-\alpha)$$
